<a href="https://colab.research.google.com/github/aryanshu/Mini-google/blob/master/query_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
cd "/gdrive/My Drive/Colab Notebooks"

/gdrive/My Drive/Colab Notebooks


In [0]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.5M      0  0:00:13  0:00:13 --:--:-- 12.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.7M      0  0:00:13  0:00:13 --:--:-- 12.3M


In [0]:
!mkdir GloVe
!curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip GloVe/glove.840B.300d.zip -d GloVe/
!mkdir fastText
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

In [0]:
from models import InferSent
import torch
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:

# Keep it on CPU or put it on GPU
use_cuda = True
model = model.cuda() if use_cuda else model

In [0]:
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.read_csv('live_tweet.csv')
df.head()

,Unnamed: 0,tweets
0,0,@RaniaKhalek People should understand that vir...
1,1,RT @bainjal: Leaked audio of Finance Minister ...
2,2,RT @drharshvardhan: Don’t chase the Sanitizer\...
3,3,RT @DrAMSinghvi: Bill moved by me in the Rajya...
4,4,@F1 can we have an event hosted in India under...


In [0]:
len(df)

20056

In [0]:
sentences=df['tweets'].tolist()

In [0]:
model.build_vocab(sentences, tokenize=True)

Found 14542(/28001) words with w2v vectors
Vocab size : 14542


In [0]:
query=input()
sent2=[query]

Delhi riots


In [0]:
sent2

['Delhi riots']

In [0]:
model.update_vocab(sent2)

Found 0(/1) words with w2v vectors
New vocab size : 4533 (added 0 words)


In [0]:
embeddings = model.encode(sentences, bsize=128)
print('nb sentences encoded : {0}'.format(len(embeddings)))

nb sentences encoded : 20056


In [0]:
#embeddings = model.encode(sentences, bsize=128, tokenize=False, verbose=True)
#print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 29723/39403 (75.4%)
Speed : 37.6 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 2000


In [0]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [0]:
embeddings[1]

array([0.05731931, 0.13585961, 0.11496196, ..., 0.        , 0.08438367,
       0.07925383], dtype=float32)

In [0]:
query2=model.encode(sent2)[0]
len(query2)

4096

In [0]:
import time
start=time.time() 
label=[]
query2=model.encode(sent2)[0]
for i in range(len(embeddings)):
  
  r=cosine(query2,embeddings[i])
  label.append([i,r])
  

In [0]:
label[:10]

[[0, 0.10443588324728266],
 [1, 0.24265742600068482],
 [2, 0.08106836554533055],
 [3, 0.20437415074940335],
 [4, 0.22912096199077736],
 [5, 0.04433247786417467],
 [6, 0.12426184153807181],
 [7, 0.21790849073462232],
 [8, 0.10261531039362988],
 [9, 0.31183878629741285]]

In [0]:
label=sorted(label,key=lambda l:l[1], reverse=True)


In [0]:

label[:25]

[[437, 0.39349751549451395],
 [1356, 0.36132332373740994],
 [99, 0.331307313532439],
 [482, 0.3309579759729943],
 [539, 0.3309579759729943],
 [882, 0.3309579759729943],
 [966, 0.3309579759729943],
 [1064, 0.3309579759729943],
 [1506, 0.3309579759729943],
 [1062, 0.33001788587372743],
 [700, 0.3300178841825376],
 [866, 0.3300178841825376],
 [1707, 0.3300178841825376],
 [78, 0.32879012965300924],
 [1292, 0.32728869515950587],
 [736, 0.32220124408816625],
 [1976, 0.319308965642642],
 [175, 0.31421526358113555],
 [1258, 0.31421526358113555],
 [1538, 0.31421526358113555],
 [1933, 0.31421526358113555],
 [9, 0.31183878629741285],
 [432, 0.31183878629741285],
 [1270, 0.31183878629741285],
 [1285, 0.31183878629741285]]

In [0]:
index_2=[i[1] for i in label[:10]]

In [0]:
import statistics 
x = statistics.mean(index_2)
print(x)

0.34018938944760563


In [0]:
index=[i[0] for i in label[:1000]]


In [0]:
len(index)

1000

In [0]:
index

In [0]:
n_tweet=[]
for r,i in enumerate(index):
  n_tweet.append(df['tweets'][i])
  

In [0]:
def my_function(x):
  return list(dict.fromkeys(x))

mylist = my_function(n_tweet)
mylist=mylist[:10]

In [0]:
len(mylist)

10

In [0]:
mylist

['RT @AapDoctors: List of 51 Primary &amp; 56 Addl Govt Labs for Corona Virus testing actoss India. Nasal/Oral Swab to be tested by RT-PCR method…',
 'RT @AnjaliB_: Hailstorm in Delhi. In light of poor weather conditions and threat of corona virus, Delhi government should provide appropria…',
 'RT @jgopikrishnan70: Pappu Diaries seized by Income Tax creating panic to Congress leadership in Delhi and Chhattisgarh https://t.co/pBOoGS…',
 'RT @OpIndia_com: Coronavirus in India: Home ministry announces ex-gratia amount of Rs 4 lakhs for the kins of patients killed due to Covid-…',
 'Why Pune is reporting an alarming number of cases? Pune is the epicentre of corona virus cases in a India. Uddhav T… https://t.co/7WH1jodyiK',
 'RT @sagarikaghose: India 5th country to isolate #COVIDー19 virus &amp; 10 people have been cured. Working in v difficult conditions, India’s val…',
 'RT @TOIIndiaNews: Five more coronavirus patients in Pune area; Maharashtra count 31 https://t.co/Z1d4gQDFm1',
 'RT @Riju

In [0]:
print('Query: {}\n '.format(str(query)))
print('Most similar tweets \n')
for r,i in enumerate(mylist):
  
  print('tweet no={}'.format(r+1))
  print(i)
  print('\n')

Query: Corona Virus Cure and Victims in Delhi
 
Most similar tweets 

tweet no=1
RT @AapDoctors: List of 51 Primary &amp; 56 Addl Govt Labs for Corona Virus testing actoss India. Nasal/Oral Swab to be tested by RT-PCR method…


tweet no=2
RT @AnjaliB_: Hailstorm in Delhi. In light of poor weather conditions and threat of corona virus, Delhi government should provide appropria…


tweet no=3
RT @jgopikrishnan70: Pappu Diaries seized by Income Tax creating panic to Congress leadership in Delhi and Chhattisgarh https://t.co/pBOoGS…


tweet no=4
RT @OpIndia_com: Coronavirus in India: Home ministry announces ex-gratia amount of Rs 4 lakhs for the kins of patients killed due to Covid-…


tweet no=5
Why Pune is reporting an alarming number of cases? Pune is the epicentre of corona virus cases in a India. Uddhav T… https://t.co/7WH1jodyiK


tweet no=6
RT @sagarikaghose: India 5th country to isolate #COVIDー19 virus &amp; 10 people have been cured. Working in v difficult conditions, India’s val…


In [0]:
type(embeddings)

numpy.ndarray

In [0]:
from numpy import asarray
from numpy import savetxt
savetxt('embeddings.csv', embeddings, delimiter=',')

In [0]:
from numpy import loadtxt
embeddings_2 = loadtxt('embeddings.csv', delimiter=',')


In [0]:
embeddings=embeddings_2

In [0]:
embeddings_2==embeddings

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [0]:
for r,i in enumerate(mylist):
  
  print('tweet no={}'.format(r+1))


In [0]:
str('hi')==str('hi')

True

In [0]:
with open('file.txt', 'w') as f:
    for item in mylist:
        f.write("%s\n" % item)
f.close()